In [1]:
import csv
import json
from pathlib import Path

import jsonlines

from config import TLCPaths
from src.data_loading import annotations, load_tlc_samples
from src.preprocessing import cistem

In [2]:
with open(TLCPaths.project_data_path.joinpath('annotate_tlc/samples_to_validate.json'), 'r') as fp:
    annotated_mentions = json.load(fp)
    print(len(annotated_mentions))

with open(TLCPaths.project_data_path.joinpath('annotate_tlc/samples_to_annotate.json'), 'r') as fp:
    annotated_mentions.extend(json.load(fp))
    print(len(annotated_mentions))


def get_cui_for_stem(stemmed_mention):
    for mention in annotated_mentions:
        if mention["stem"] == stemmed_mention:
            return mention["cui"]


def get_annotation_by_id(id):
    for annotation in annotations:
        if annotation.id == id:
            return annotation


for m in annotated_mentions:
    if m["stem"] == "nierenstei":
        print(m)

672
998
{'cui': ['C0022650'], 'source': ['WIKTIONARY'], 'language': ['GER'], 'name': ['nierenstein'], 'index_term': ['nierenstei'], 'id': '477d2407-cf96-4bd0-8603-7b1978961703', '_version_': 1760194257048567815, 'score': 1.0, 'stem': 'nierenstei'}


In [233]:
# using solr high confidence matches
with open(TLCPaths.project_data_path.joinpath('search_terms_single_and_ids.json'), 'r') as fp:
    terms_and_ids = json.load(fp)

ids_and_terms = {value: [key] for key in terms_and_ids for value in terms_and_ids[key]}

In [218]:
tTLC_path = Path("/home/tim/MedicalLay/datasets/TLC_translated")
for file in tTLC_path.iterdir():
    print(file)

/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_texts_ger_eng_translation.jsonl
/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_with_entity_markers.jsonl
/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_with_entity_markers_v2.jsonl
/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_texts_eng_ger_translation.jsonl


In [219]:
translated_texts = {}
with jsonlines.open(
        "/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_texts_ger_eng_translation.jsonl") as reader:
    for obj in reader:
        file = obj["meta"]["id"]
        translated_texts[file] = obj["meta"]["translated_sentence"]

In [220]:
entity_markers = {}
with jsonlines.open(
        "/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_with_entity_markers_v2.jsonl") as reader:
    for obj in reader:
        entity_types = obj['meta']['entity_types']
        sorted_entity_types = sorted(entity_types, key=lambda x: list(x.keys())[0])
        entity_markers[obj["id"]] = {list(d.keys())[0]: d[list(d.keys())[0]] for d in
                                     sorted_entity_types}

In [221]:
print(len(translated_texts), len(entity_markers))

572 3995


In [222]:
for file1 in translated_texts:
    if file1 not in entity_markers:
        print(file1)

for file2 in entity_markers:
    if file2 not in translated_texts:
        print(file2)

file_0164_Stephannie0907_2017-09-04_
file_0339_ChrisFifty_2017-09-27_
file_0575_blackmouse88_2015-06-07_
file_0576_Schneehexe_2015-06-07_
file_0577_blackmouse88_2015-06-07_
file_0578_danae87_2017-07-10_
file_0579_Loroeffiyu_2017-07-10_
file_0580_danae87_2017-07-10_
file_0581_Loroeffiyu_2017-07-10_
file_0582_danae87_2017-07-10_
file_0583_pieplant_2017-07-10_
file_0584_Loroeffiyu_2017-07-10_
file_0585_danae87_2017-07-10_
file_0586_Fizzlypuzzly_2017-07-10_
file_0587_pieplant_2017-07-10_
file_0588_Loroeffiyu_2017-07-10_
file_0589_danae87_2017-07-10_
file_0590_Loroeffiyu_2017-07-10_
file_0591_danae87_2017-07-10_
file_0592_luomaren_2017-07-10_
file_0593_Elly28.3._2015-06-11_
file_0594_musicus_65_2015-06-11_
file_0595_Seanet7or_2015-06-12_
file_0596_carmen11211970_2016-04-15_
file_0597_Nike_Zoe_2015-06-13_
file_0598_Schneehexe_2015-06-13_
file_0599_Nike_Zoe_2015-06-13_
file_0600_quengler_2015-06-20_
file_0601_Schneehexe_2015-06-21_
file_0602_Garou78_2015-06-21_
file_0603_quengler_2015-06-21_


In [223]:
# count number of markers and compare with tlc original annotation
mean_markers = 0
for file, markers in entity_markers.items():
    mean_markers += len(markers)
print("mean markers per file is ", mean_markers / len(entity_markers))

mean markers per file is  2.2590738423028784


In [224]:
samples = load_tlc_samples()

mean_annotations = 0
for sample in samples:
    mean_annotations += len(sample.annotations)
print("mean annotations per sample is ", mean_annotations / len(samples))

mean annotations per sample is  1.8493493493493494


In [225]:
print(list(translated_texts.keys()))

['file_0001_RockVision_2018-08-28_', 'file_0002_BenitaB._2018-08-28_', 'file_0003_Myrmidone_2018-08-28_', 'file_0004_Luci32_2018-08-28_', 'file_0005_RockVision_2018-08-28_', 'file_0006_Luci32_2018-08-28_', 'file_0007_pelztier86_2018-08-28_', 'file_0008_RockVision_2018-08-28_', 'file_0009_bolit_2018-08-28_', 'file_0010_RockVision_2018-08-28_', 'file_0011_Myrmidone_2018-08-28_', 'file_0012_RockVision_2018-08-28_', 'file_0013_Myrmidone_2018-09-25_', 'file_0014_RockVision_2018-10-08_', 'file_0015_Nike_Zoe_2018-10-08_', 'file_0016_Ceyda50_2018-01-10_', 'file_0017_Myrmidone_2018-01-10_', 'file_0018_Ceyda50_2018-01-11_', 'file_0019_Myrmidone_2018-01-11_', 'file_0020_Simone13_2018-01-21_', 'file_0021_BenitaB._2018-01-26_', 'file_0022_Simone13_2018-01-27_', 'file_0023_BenitaB._2018-02-04_', 'file_0024_0815user_2018-02-04_', 'file_0025_0815user_2018-02-04_', 'file_0026_Simone13_2018-02-04_', 'file_0027_Moehrensuppe_2018-01-19_', 'file_0028_jeri_2018-01-19_', 'file_0029_Keendary_2018-01-24_', 'fi

In [226]:
file_name = 'file_0571_Thia2016_2016-03-01_'
# file_name = "file_0535_Katja811_2016-03-10_"
# file_name = 'file_0573_Nowheregirl_2016-04-27_'
file_name = 'file_0557_katha26_2017-03-05_'
print(translated_texts[file_name], "\n\n", entity_markers[file_name])

Gross, may I ask how this happened at such a young age? I have a <T14>congenital</T14> kidney disease. Don't you have a chance to get your kidney? Or did the transplant go wrong? I got a kidney from my mother when I was 16 and it worked great for 10 years. Then I <T15>caught one infection after another</T15> after <T6>surgery</T6>, which eventually \killed the donor kidney. I don't want another transplant at this time for personal reasons, but I am on the waiting list (but sort of \paused\). Do you do night<T7><T9>dialysis</T7></T9>? So here in our place there is the possibility to go to <T2><T10>dialysis</T2></T10> at night, so you still have the days off. Yes, I do night<T3><T11>dialysis</T3></T11>, so I can at least still work part-time and still have time for hobbies etc. I get off <T4><T12>dial</T4>ysis</T12> at about 7 in the morning and then sleep at home for another 2 to 3 hours, after that I am fine. But a life with <T5><T13>dialysis</T5></T13> requires a lot of discipline, an

In [227]:
# tag in translation -> german term -> stemmed german term -> cui found by solr

In [13]:
import re

tag = "T2"
re.findall("<" + tag + ">(.*?)</" + tag + ">", translated_texts[file_name])

['stone']

In [239]:
def remove_tags(text):
    pattern = r"<[^>]+>"
    return re.sub(pattern, "", text)


cuis = {}
for file, translated_text in translated_texts.items():
    markers = entity_markers[file]
    cuis[file] = {}
    for tag, annotation in markers.items():
        translated_mention = re.findall("<" + tag + ">(.*?)</" + tag + ">", translated_text)
        translated_mention = " ".join(translated_mention)
        translated_mention = remove_tags(translated_mention)
        if "Laienbegriff" in annotation:
            mention = annotation["Laienbegriff"]
        elif "Fachterm" in annotation:
            mention = annotation["Fachterm"]
        else:
            continue
        print(mention)
        # this should use the terms and ids and map to the corresponding annotation group
        # original_annotation = next(ann for ann in annotations if ann.get_mention() == mention.strip())
        stemmed_mention = cistem.stem(mention).strip()
        orig_ann_id = ids_and_terms[stemmed_mention]
        # cui = get_cui_for_stem(stemmed_mention)
        cuis[file][tag] = dict(translated_mention=translated_mention, mention=mention,
                               stemmed_mention=stemmed_mention, cui=cui)

    # dict(translated_mention= , )

hoher Blutdruck


KeyError: 'hoher blutdruck'

In [235]:
annotations

[Annotation(tech_term=None, lay_term='Blähungen', type=<TermType.LAY: 'LAY'>, span_start=86, span_end=95, synonyms=['Flatulenzen', 'Flatulenzen'], id=6992),
 Annotation(tech_term=None, lay_term='bauchspeicheldrüse', type=<TermType.LAY: 'LAY'>, span_start=194, span_end=212, synonyms=['Pankreas', 'Pankreas'], id=6993),
 Annotation(tech_term='Nierenbeckenentzündung', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=30, span_end=52, synonyms=['Pyelonephritis', 'Pyelonephritis'], id=3633),
 Annotation(tech_term=None, lay_term='Verkleinerte NIeren', type=<TermType.LAY: 'LAY'>, span_start=9, span_end=28, synonyms=['Schrumpfniere (Nephrozirrhose)', 'Schrumpfniere'], id=2472),
 Annotation(tech_term='Gallensteine', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=121, span_end=133, synonyms=['Cholelith', 'Cholelith'], id=6202),
 Annotation(tech_term=None, lay_term='Gallensteine in den Gallengängen', type=<TermType.LAY: 'LAY'>, span_start=288, span_end=320, synonyms=['Choledocholith

In [172]:
cuis[file_name]

{'T1': {'translated_mention': '',
  'mention': 'Dialyse',
  'stemmed_mention': 'dialy',
  'cui': ['C0011945']},
 'T10': {'translated_mention': 'dialysis',
  'mention': 'Dialyse',
  'stemmed_mention': 'dialy',
  'cui': ['C0011945']},
 'T11': {'translated_mention': 'dialysis',
  'mention': 'dialyse',
  'stemmed_mention': 'dialy',
  'cui': ['C0011945']},
 'T12': {'translated_mention': 'dialysis',
  'mention': 'Dialyse',
  'stemmed_mention': 'dialy',
  'cui': ['C0011945']},
 'T13': {'translated_mention': 'dialysis',
  'mention': 'Dialyse',
  'stemmed_mention': 'dialy',
  'cui': ['C0011945']},
 'T14': {'translated_mention': 'congenital',
  'mention': 'angeborene',
  'stemmed_mention': 'angebor',
  'cui': None},
 'T15': {'translated_mention': 'caught one infection after another',
  'mention': 'eine Infektion nach der anderen eingefangen',
  'stemmed_mention': 'eine infektion nach der anderen eingefang',
  'cui': None},
 'T2': {'translated_mention': 'dialysis',
  'mention': 'Dialyse',
  'stem

In [173]:
translated_mentions_and_cuis = []
for file, tags in cuis.items():
    for tag, mention_info in tags.items():
        if mention_info["cui"]:
            mention_info["file"] = file
            mention_info["tag"] = tag
            mention_info["cui"] = mention_info["cui"]
            translated_mentions_and_cuis.append(mention_info)
print(len(translated_mentions_and_cuis))

940


In [ ]:
translated_mentions_and_cuis = [info for info in translated_mentions_and_cuis if
                                info["translated_mention"]]
# number of translated mentions for that a cui was found and that were correctly translated
print(len(translated_mentions_and_cuis))#%% raw


In [ ]:
import csv
import json
from pathlib import Path

import jsonlines

from config import TLCPaths
from src.data_loading import annotations, load_tlc_samples
from src.preprocessing import cistem

In [ ]:
with open(TLCPaths.project_data_path.joinpath('annotate_tlc/samples_to_validate.json'), 'r') as fp:
    annotated_mentions = json.load(fp)
    print(len(annotated_mentions))

with open(TLCPaths.project_data_path.joinpath('annotate_tlc/samples_to_annotate.json'), 'r') as fp:
    annotated_mentions.extend(json.load(fp))
    print(len(annotated_mentions))


def get_cui_for_stem(stemmed_mention):
    for mention in annotated_mentions:
        if mention["stem"] == stemmed_mention:
            return mention["cui"]


def get_annotation_by_id(id):
    for annotation in annotations:
        if annotation.id == id:
            return annotation


for m in annotated_mentions:
    if m["stem"] == "nierenstei":
        print(m)

In [ ]:
# using solr high confidence matches
with open(TLCPaths.project_data_path.joinpath('search_terms_single_and_ids.json'), 'r') as fp:
    terms_and_ids = json.load(fp)

ids_and_terms = {value: [key] for key in terms_and_ids for value in terms_and_ids[key]}
# 
# 
# mentions_with_cuis = {}
# no_cui_for_group = 0
# for stem, annotation_ids in terms_and_ids.items():
#     cui = get_cui_for_stem(stem)
#     if not cui:
#         no_cui_for_group += 1
#         continue    
#     grouped_annotations = [get_annotation_by_id(id) for id in annotation_ids]
#     for annotation in grouped_annotations:
#         mention = annotation.get_mention()
#         mentions_with_cuis[mention] = cui
# print(no_cui_for_group, len(terms_and_ids))

In [ ]:
tTLC_path = Path("/home/tim/MedicalLay/datasets/TLC_translated")
for file in tTLC_path.iterdir():
    print(file)

In [ ]:
translated_texts = {}
with jsonlines.open(
        "/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_texts_ger_eng_translation.jsonl") as reader:
    for obj in reader:
        file = obj["meta"]["id"]
        translated_texts[file] = obj["meta"]["translated_sentence"]

In [ ]:
entity_markers = {}
with jsonlines.open(
        "/home/tim/MedicalLay/datasets/TLC_translated/TLC_v01_with_entity_markers_v2.jsonl") as reader:
    for obj in reader:
        entity_types = obj['meta']['entity_types']
        sorted_entity_types = sorted(entity_types, key=lambda x: list(x.keys())[0])
        entity_markers[obj["id"]] = {list(d.keys())[0]: d[list(d.keys())[0]] for d in
                                     sorted_entity_types}

In [ ]:
print(len(translated_texts), len(entity_markers))

In [ ]:
for file1 in translated_texts:
    if file1 not in entity_markers:
        print(file1)

for file2 in entity_markers:
    if file2 not in translated_texts:
        print(file2)

In [ ]:
# count number of markers and compare with tlc original annotation
mean_markers = 0
for file, markers in entity_markers.items():
    mean_markers += len(markers)
print("mean markers per file is ", mean_markers / len(entity_markers))

In [ ]:
samples = load_tlc_samples()

mean_annotations = 0
for sample in samples:
    mean_annotations += len(sample.annotations)
print("mean annotations per sample is ", mean_annotations / len(samples))

In [ ]:
print(list(translated_texts.keys()))

In [ ]:
file_name = 'file_0571_Thia2016_2016-03-01_'
# file_name = "file_0535_Katja811_2016-03-10_"
# file_name = 'file_0573_Nowheregirl_2016-04-27_'
file_name = 'file_0557_katha26_2017-03-05_'
print(translated_texts[file_name], "\n\n", entity_markers[file_name])

In [ ]:
# tag in translation -> german term -> stemmed german term -> cui found by solr

In [ ]:
import re

tag = "T2"
re.findall("<" + tag + ">(.*?)</" + tag + ">", translated_texts[file_name])

In [ ]:
def remove_tags(text):
    pattern = r"<[^>]+>"
    return re.sub(pattern, "", text)


cuis = {}
for file, translated_text in translated_texts.items():
    markers = entity_markers[file]
    cuis[file] = {}
    for tag, annotation in markers.items():
        translated_mention = re.findall("<" + tag + ">(.*?)</" + tag + ">", translated_text)
        translated_mention = " ".join(translated_mention)
        translated_mention = remove_tags(translated_mention)
        if "Laienbegriff" in annotation:
            mention = annotation["Laienbegriff"]
        elif "Fachterm" in annotation:
            mention = annotation["Fachterm"]
        else:
            continue
        print(mention)
        # this should use the terms and ids and map to the corresponding annotation group
        # original_annotation = next(ann for ann in annotations if ann.get_mention() == mention.strip())
        stemmed_mention = cistem.stem(mention).strip()
        orig_ann_id = ids_and_terms[stemmed_mention]
        # cui = get_cui_for_stem(stemmed_mention)
        cuis[file][tag] = dict(translated_mention=translated_mention, mention=mention,
                               stemmed_mention=stemmed_mention, cui=cui)

    # dict(translated_mention= , )

In [ ]:
annotations

In [ ]:
cuis[file_name]

In [ ]:
translated_mentions_and_cuis = []
for file, tags in cuis.items():
    for tag, mention_info in tags.items():
        if mention_info["cui"]:
            mention_info["file"] = file
            mention_info["tag"] = tag
            mention_info["cui"] = mention_info["cui"]
            translated_mentions_and_cuis.append(mention_info)
print(len(translated_mentions_and_cuis))

In [ ]:
translated_mentions_and_cuis = [info for info in translated_mentions_and_cuis if
                                info["translated_mention"]]
# number of translated mentions for that a cui was found and that were correctly translated
print(len(translated_mentions_and_cuis))    

In [ ]:
import nltk
ps = nltk.stem.PorterStemmer()

In [ ]:
print("number of samples:", len(samples))
print("number of mentions: ", len(annotations))
mentions = [annotation.get_mention() for annotation in annotations]
stemmed_mentions = [cistem.stem(mention).strip() for mention in mentions]
print("number of unique mentions:", len(set(stemmed_mentions)))
stemmed_mentions_with_cui = [stemmed_mention for stemmed_mention in stemmed_mentions if get_cui_for_stem(stemmed_mention)]
print("number of mentions with cui found by Solr with high confidence:", len(stemmed_mentions_with_cui))
print("number of unique mentions with cui found by Solr with high confidence:", len(set(stemmed_mentions_with_cui)))
print("number of translated samples:", len(translated_texts))

# remove empty translations
translated_mentions = []
for file, tags in cuis.items():
    for tag, mention_info in tags.items():
        if mention_info["translated_mention"]:
            translated_mentions.append(mention_info["translated_mention"])
            
print("number of translated mentions:", len(translated_mentions))
print("number of unique translated mentions:", len(set([ps.stem(mention).strip() for mention in translated_mentions])))
print("number of translated mentions for that a cui was found:", len(translated_mentions_and_cuis))

# number of unique translated mentions with cui
unique_stemmed_translated_mentions = set()
unique_translated_mentions_and_cuis = []
for info in translated_mentions_and_cuis:
    stemmed_translated_mention = ps.stem(info["translated_mention"]).strip()
    if stemmed_translated_mention not in unique_stemmed_translated_mentions:
        info["stemmed_translated_mention"] = stemmed_translated_mention
        unique_translated_mentions_and_cuis.append(info)
        unique_stemmed_translated_mentions.add(stemmed_translated_mention)
        

print("number of unique translated mentions with cui:", len(unique_translated_mentions_and_cuis))

In [ ]:
with open(TLCPaths.project_data_path.joinpath("english_mentions_and_labels.csv"),"w", newline="") as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for translated_mention_info in translated_mentions_and_cuis:
        spamwriter.writerow([translated_mention_info["translated_mention"], translated_mention_info["cui"][0]])

In [ ]:
translated_mentions_and_cuis[0]#%%
import nltk
ps = nltk.stem.PorterStemmer()

In [176]:
print("number of samples:", len(samples))
print("number of mentions: ", len(annotations))
mentions = [annotation.get_mention() for annotation in annotations]
stemmed_mentions = [cistem.stem(mention).strip() for mention in mentions]
print("number of unique mentions:", len(set(stemmed_mentions)))
stemmed_mentions_with_cui = [stemmed_mention for stemmed_mention in stemmed_mentions if get_cui_for_stem(stemmed_mention)]
print("number of mentions with cui found by Solr with high confidence:", len(stemmed_mentions_with_cui))
print("number of unique mentions with cui found by Solr with high confidence:", len(set(stemmed_mentions_with_cui)))
print("number of translated samples:", len(translated_texts))

# remove empty translations
translated_mentions = []
for file, tags in cuis.items():
    for tag, mention_info in tags.items():
        if mention_info["translated_mention"]:
            translated_mentions.append(mention_info["translated_mention"])
            
print("number of translated mentions:", len(translated_mentions))
print("number of unique translated mentions:", len(set([ps.stem(mention).strip() for mention in translated_mentions])))
print("number of translated mentions for that a cui was found:", len(translated_mentions_and_cuis))

# number of unique translated mentions with cui
unique_stemmed_translated_mentions = set()
unique_translated_mentions_and_cuis = []
for info in translated_mentions_and_cuis:
    stemmed_translated_mention = ps.stem(info["translated_mention"]).strip()
    if stemmed_translated_mention not in unique_stemmed_translated_mentions:
        info["stemmed_translated_mention"] = stemmed_translated_mention
        unique_translated_mentions_and_cuis.append(info)
        unique_stemmed_translated_mentions.add(stemmed_translated_mention)
        

print("number of unique translated mentions with cui:", len(unique_translated_mentions_and_cuis))

number of samples: 3996
number of mentions:  7390
number of unique mentions: 1468
number of mentions with cui found by Solr with high confidence: 2610
number of unique mentions with cui found by Solr with high confidence: 240
number of translated samples: 572
number of translated mentions: 1767
number of unique translated mentions: 690
number of translated mentions for that a cui was found: 762
number of unique translated mentions with cui: 169


In [179]:
with open(TLCPaths.project_data_path.joinpath("english_mentions_and_labels.csv"),"w", newline="") as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for translated_mention_info in translated_mentions_and_cuis:
        spamwriter.writerow([translated_mention_info["translated_mention"], translated_mention_info["cui"][0]])

In [180]:
translated_mentions_and_cuis[0]

{'translated_mention': 'protein',
 'mention': 'Eiweiß',
 'stemmed_mention': 'eiweiss',
 'cui': ['C0033684'],
 'file': 'file_0001_RockVision_2018-08-28_',
 'tag': 'T2',
 'stemmed_translated_mention': 'protein'}